In [ ]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
ls

'Colab Notebooks'/  'My image model.tm'   submission.csv
 data_loader_v2/     open.zip


In [ ]:
!unzip open.zip

Archive:  open.zip
  inflating: sample_submission.csv   
  inflating: test_x.csv              
  inflating: train.csv               


In [ ]:
import pandas as pd
import numpy as np
import torch
import random
import torch.nn

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [ ]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test_x.csv")
submit=pd.read_csv("sample_submission.csv")

In [ ]:
train=train.drop(labels='index',axis=1)
test=test.drop(labels='index',axis=1)

In [ ]:
train.to_csv("train_data.csv",index=False)
test.to_csv("test_data.csv",index=False)

In [ ]:
train_data=pd.read_csv("train_data.csv")
train_data

,text,author
0,"He was almost choking. There was so much, so m...",3
1,"“Your sister asked for it, I suppose?”",2
2,"She was engaged one day as she walked, in per...",1
3,"The captain was in the porch, keeping himself ...",4
4,"“Have mercy, gentlemen!” odin flung up his han...",3
...,...,...
54874,"“Is that you, Mr. Smith?” odin whispered. “I h...",2
54875,"I told my plan to the captain, and between us ...",4
54876,"""Your sincere well-wisher, friend, and sister...",1
54877,“Then you wanted me to lend you money?”,3


# 자연어 처리 과정



1.   전처리

    불용어 제거, 형태소 분석, 표제어 추출

2.   Vectorization(수치화)=embedding(?)

    - one-hot-encoding 
    - Tfidf
    - 단어 간 유사도 고려하는 embedding
      - Word2vec, fasttext,Glove 등

4. Modeling

    보통 RNN 모델 사용(GRU, LSTM)


이번에 사용해 볼 거
- 전처리

    - 특수문자제거
    - 형태소 분석
    - 불용어 제거
- Tf-idf, padding
- 사전학습된 Embedding
- LSTM 모델

## 데이콘 예시-한국어 전처리

  - konlpy

    한국어 정보처리를 위한 파이썬 패키지

    pos (품사태깅): 형태소의 뜻과 문맥을 고려하여 맞게 식별하는 것

    예) ~에 => JKM
    


In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 92kB 10.0MB/s 
     |████████████████████████████████| 460kB 29.9MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
from konlpy.tag import Okt
import re

def text_preprocessing(text_list):
    
    stopwords = ['을', '를', '이', '가', '은', '는', 'null','께서'] #불용어 설정
    tokenizer = Okt() #형태소 분석기 
    token_list = []
    
    for text in text_list:
        txt = re.sub('[^가-힣a-z]', ' ', text) #한글과 영어 소문자만 남기고 다른 글자 모두 제거
        token = tokenizer.morphs(txt) #형태소 분석
        token = [t for t in token if t not in stopwords or type(t) != float] #형태소 분석 결과 중 stopwords에 해당하지 않는 것만 추출
        token_list.append(token)
        
    return token_list, tokenizer

#형태소 분석기를 따로 저장한 이유는 후에 test 데이터 전처리를 진행할 때 이용해야 되기 때문입니다. 
train['new_article'], okt = text_preprocessing(train['content']) 

# torchtext

  - 앞서 말한 과정들을 한번에 진행해주는 편리함 제공

    (토큰화, 단어 집합 생성, 정수 인코딩, 패딩 등등)


In [ ]:
!pip install torchtext

In [ ]:
#tokenizer로 Spacy를 사용할 것
#이외에도 NLTK, 영어는 띄어쓰기만으로도 tokenizer가 잘 됨 (한국어는 아님)
import spacy

spacy_en=spacy.load('en')

In [ ]:
from torchtext import data


#필드 정의

TEXT=data.Field(sequential=True,use_vocab=True,tokenize="spacy",
                lower=True,batch_first=True,fix_length=1000)

LABEL=data.Field(sequential=False,use_vocab=False,
                lower=False,batch_first=True,is_target=True,
                 #preprocessing= lambda x: int (x)
                 )

##토큰화(한국어로 치면 형태소 화)

In [ ]:
from torchtext.data import TabularDataset


train_data, test_data = TabularDataset.splits(
        path='.', train='train_data.csv', test='test_data.csv', format='csv',
        fields=[('text', TEXT), ('label', LABEL)], skip_header=True)

In [ ]:
print(vars(train_data[0]))
print(vars(test_data[0]))

{'text': ['he', 'was', 'almost', 'choking', '.', 'there', 'was', 'so', 'much', ',', 'so', 'much', 'he', 'wanted', 'to', 'say', ',', 'but', 'strange', 'exclamations', 'were', 'all', 'that', 'came', 'from', 'his', 'lips', '.', 'the', 'pole', 'gazed', 'fixedly', 'at', 'him', ',', 'at', 'the', 'bundle', 'of', 'notes', 'in', 'his', 'hand', ';', 'looked', 'at', 'odin', ',', 'and', 'was', 'in', 'evident', 'perplexity', '.'], 'label': '3'}
{'text': ['“', 'not', 'at', 'all', '.', 'i', 'think', 'she', 'is', 'one', 'of', 'the', 'most', 'charming', 'young', 'ladies', 'i', 'ever', 'met', ',', 'and', 'might', 'have', 'been', 'most', 'useful', 'in', 'such', 'work', 'as', 'we', 'have', 'been', 'doing', '.', 'she', 'had', 'a', 'decided', 'genius', 'that', 'way', ':', 'witness', 'the', 'way', 'in', 'which', 'she', 'preserved', 'that', 'agra', 'plan', 'from', 'all', 'the', 'other', 'papers', 'of', 'her', 'father', '.', 'but', 'love', 'is', 'an', 'emotional', 'thing', ',', 'and', 'whatever', 'is', 'emotio

##단어 집합 만들기

- 자주나오는 단어의 집합을 만들어 숫자와 매핑
- 단어의 개수 지정 가능

In [ ]:
TEXT.build_vocab(train_data, min_freq=10, max_size=10000)
#10번이상 안나오면 단어가 아님, 단어장의 최대 크기는 10000

In [ ]:
print('단어 집합의 크기 : {}'.format(len(TEXT.vocab)))

단어 집합의 크기 : 9702


In [ ]:
print(TEXT.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7fee2fc03b00>, {'<unk>': 0, '<pad>': 1, ',': 2, '.': 3, 'the': 4, 'and': 5, 'to': 6, 'of': 7, 'i': 8, 'a': 9, 'odin': 10, 'in': 11, '“': 12, 'he': 13, '”': 14, 'you': 15, 'it': 16, 'that': 17, 'was': 18, 'his': 19, 'with': 20, ';': 21, 'had': 22, 'not': 23, 'for': 24, 'as': 25, '?': 26, '"': 27, 'at': 28, 'her': 29, 'my': 30, 'but': 31, 'is': 32, '!': 33, 'have': 34, 'be': 35, 'she': 36, 'me': 37, 'him': 38, 'said': 39, 'on': 40, 'all': 41, '-': 42, 'so': 43, "'": 44, 'this': 45, 'what': 46, 'do': 47, 'there': 48, '’': 49, 'by': 50, '‘': 51, 'from': 52, 'no': 53, '’s': 54, 'which': 55, 'were': 56, 'we': 57, 'one': 58, 'they': 59, 'would': 60, 'if': 61, 'been': 62, 'are': 63, 'your': 64, 'an': 65, 'very': 66, 'could': 67, "'s": 68, 'mr.': 69, 'when': 70, 'will': 71, 'out': 72, '--': 73, 'or': 74, 'up': 75, ' ': 76, 'man': 77, 'upon': 78, 'them': 79, 'now': 80, 'more': 81, 'who': 82, '_': 83, 'then': 84, 'some': 85, 'did': 86, 'am': 87, 'know

 < unk > : 0  =>단어집합에 없는 단어를 표현할 때 사용

 < pad > : 1 => 길이를 맞추는 패딩 작업을 할 때 사용 , 즉 빈자리는 1로 채움

##데이터로더 만들기

In [ ]:
from torchtext.data import Iterator

batch_size=10

train_loader=Iterator(dataset=train_data,batch_size=batch_size,device="cuda")
test_loader=Iterator(dataset=test_data,batch_size=batch_size,device="cuda")

In [ ]:
len(train_loader),len(test_loader)

(5488, 1962)

##단어간 유사도를 가지는 워드 embedding으로 vetorize하기

[참고]https://heegyukim.medium.com/keras-embedding%EC%9D%80-word2vec%EC%9D%B4-%EC%95%84%EB%8B%88%EB%8B%A4-619bd683ded6


1.   단순 embedding layer를 넣어서 모델을 학습하면서 워드 embedding 하기

    ***이 경우에는 단어간 유사도를 가지는 워드 embedding 방법이 아님***
2.   사전학습된 embedding을 불러와서 embedding 하기
  - Word2Vec, Glove, Fasttext
  - 사전학습시에 없었던 데이터의 경우에는 다시 학습이 필요함

3. 따라서, 사전학습된 embedding(Word2Vec 등)을 불러와서 embedding layer의 가중치 초기값으로 사용하기 

     **이미 사전학습이 된 상태에서 다시 학습을 하며 embedding 됨**

이번에는 사전학습 Glove모델을  사용할 것.

In [ ]:
from torchtext.vocab import GloVe

TEXT.build_vocab(train_data, vectors=GloVe(name='6B', dim=300), max_size=10000, min_freq=10)
LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [07:15, 1.98MB/s]                           
100%|█████████▉| 399385/400000 [00:58<00:00, 7304.53it/s]

In [ ]:
print(TEXT.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7fee2fc03b00>, {'<unk>': 0, '<pad>': 1, ',': 2, '.': 3, 'the': 4, 'and': 5, 'to': 6, 'of': 7, 'i': 8, 'a': 9, 'odin': 10, 'in': 11, '“': 12, 'he': 13, '”': 14, 'you': 15, 'it': 16, 'that': 17, 'was': 18, 'his': 19, 'with': 20, ';': 21, 'had': 22, 'not': 23, 'for': 24, 'as': 25, '?': 26, '"': 27, 'at': 28, 'her': 29, 'my': 30, 'but': 31, 'is': 32, '!': 33, 'have': 34, 'be': 35, 'she': 36, 'me': 37, 'him': 38, 'said': 39, 'on': 40, 'all': 41, '-': 42, 'so': 43, "'": 44, 'this': 45, 'what': 46, 'do': 47, 'there': 48, '’': 49, 'by': 50, '‘': 51, 'from': 52, 'no': 53, '’s': 54, 'which': 55, 'were': 56, 'we': 57, 'one': 58, 'they': 59, 'would': 60, 'if': 61, 'been': 62, 'are': 63, 'your': 64, 'an': 65, 'very': 66, 'could': 67, "'s": 68, 'mr.': 69, 'when': 70, 'will': 71, 'out': 72, '--': 73, 'or': 74, 'up': 75, ' ': 76, 'man': 77, 'upon': 78, 'them': 79, 'now': 80, 'more': 81, 'who': 82, '_': 83, 'then': 84, 'some': 85, 'did': 86, 'am': 87, 'know

In [ ]:
#임베딩된 값
#이제 이 임베딩된 값들을 모델의 input으로 쓰면 된다!
print(TEXT.vocab.vectors[4])

tensor([ 4.6560e-02,  2.1318e-01, -7.4364e-03, -4.5854e-01, -3.5639e-02,
         2.3643e-01, -2.8836e-01,  2.1521e-01, -1.3486e-01, -1.6413e+00,
        -2.6091e-01,  3.2434e-02,  5.6621e-02, -4.3296e-02, -2.1672e-02,
         2.2476e-01, -7.5129e-02, -6.7018e-02, -1.4247e-01,  3.8825e-02,
        -1.8951e-01,  2.9977e-01,  3.9305e-01,  1.7887e-01, -1.7343e-01,
        -2.1178e-01,  2.3617e-01, -6.3681e-02, -4.2318e-01, -1.1661e-01,
         9.3754e-02,  1.7296e-01, -3.3073e-01,  4.9112e-01, -6.8995e-01,
        -9.2462e-02,  2.4742e-01, -1.7991e-01,  9.7908e-02,  8.3118e-02,
         1.5299e-01, -2.7276e-01, -3.8934e-02,  5.4453e-01,  5.3737e-01,
         2.9105e-01, -7.3514e-03,  4.7880e-02, -4.0760e-01, -2.6759e-02,
         1.7919e-01,  1.0977e-02, -1.0963e-01, -2.6395e-01,  7.3990e-02,
         2.6236e-01, -1.5080e-01,  3.4623e-01,  2.5758e-01,  1.1971e-01,
        -3.7135e-02, -7.1593e-02,  4.3898e-01, -4.0764e-02,  1.6425e-02,
        -4.4640e-01,  1.7197e-01,  4.6246e-02,  5.8

In [ ]:
#초기화 예
#나중에 모델 설계할 때 embedding_layer를 넣으면 됨
embedding_layer = torch.nn.Embedding.from_pretrained(TEXT.vocab.vectors, freeze=False)

In [ ]:
embedding_layer(torch.LongTensor([4]))

tensor([[ 4.6560e-02,  2.1318e-01, -7.4364e-03, -4.5854e-01, -3.5639e-02,
          2.3643e-01, -2.8836e-01,  2.1521e-01, -1.3486e-01, -1.6413e+00,
         -2.6091e-01,  3.2434e-02,  5.6621e-02, -4.3296e-02, -2.1672e-02,
          2.2476e-01, -7.5129e-02, -6.7018e-02, -1.4247e-01,  3.8825e-02,
         -1.8951e-01,  2.9977e-01,  3.9305e-01,  1.7887e-01, -1.7343e-01,
         -2.1178e-01,  2.3617e-01, -6.3681e-02, -4.2318e-01, -1.1661e-01,
          9.3754e-02,  1.7296e-01, -3.3073e-01,  4.9112e-01, -6.8995e-01,
         -9.2462e-02,  2.4742e-01, -1.7991e-01,  9.7908e-02,  8.3118e-02,
          1.5299e-01, -2.7276e-01, -3.8934e-02,  5.4453e-01,  5.3737e-01,
          2.9105e-01, -7.3514e-03,  4.7880e-02, -4.0760e-01, -2.6759e-02,
          1.7919e-01,  1.0977e-02, -1.0963e-01, -2.6395e-01,  7.3990e-02,
          2.6236e-01, -1.5080e-01,  3.4623e-01,  2.5758e-01,  1.1971e-01,
         -3.7135e-02, -7.1593e-02,  4.3898e-01, -4.0764e-02,  1.6425e-02,
         -4.4640e-01,  1.7197e-01,  4.